# LangChain 0.1.0 Quickstart - Create Your Own RAG Chains

Welcome to this Quickstart walkthrough of the new version of Langchain. This tutorial follows the official Langchain documentation quickstart.

# 1.0 Install the required packages

Some packages have changed inside Langchain. This is how you should install and import them from now on. Remember that langchain has been divided into three main packages:
- `langchain` contains pre-built chains and other stuff you may need.
- `langchain-core` contains the main abstractions of the library.
- `langchain-community` contains third-party integrations like Chroma, FAISS, etc.
- Some third-party integrations have their own package (outside `langchain-community`) that you should install if you want to use them. For example, OpenAI has its own package: `langchain-openai`.

All three are installed when you run `pip install langchain`. But you can install community or core individually as well.

In [ ]:
! pip install langchain
! pip install langchain-openai

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke("what is LangSmith and How to use it?")

AIMessage(content='LangSmith is a software tool that helps users to easily generate language models and deploy them for various natural language processing tasks. It provides a user-friendly interface for creating and training models using pre-built datasets or custom data. Users can fine-tune the models according to their specific requirements and deploy them for tasks such as text classification, sentiment analysis, named entity recognition, and more.\n\nTo use LangSmith, users can follow these steps:\n\n1. Install LangSmith by following the instructions provided on their website or documentation.\n2. Create a new project and import the necessary datasets or input data.\n3. Choose the type of language model you want to create (e.g., text classification, sentiment analysis).\n4. Configure the model parameters and hyperparameters according to your requirements.\n5. Train the model using the provided tools and monitor its performance.\n6. Fine-tune the model if needed and evaluate its a

# 2.0 Create your first chain with LCEL

LCEL is now the default way to create chains in LangChain. It has a more pipeline-like syntax and allows you to modify already-existing chains.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an English-Polosh translator that return whatever the user says in Polish"),
    ("user", "{input}")
])

In [5]:
chain = prompt | llm

In [6]:
chain.invoke({
    "input": "Hello, Good Morning"
    })

AIMessage(content='Cześć, Dzień Dobry.')

In [ ]:
# add output parser to the chain

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

# 3.0 Create a Retrieval Chain


## 3.1 Load the source documents

First, we will have to load the documents that will enrich our LLM prompt. We will use [this blog post](https://blog.langchain.dev/langchain-v0-1-0/) from LangChain's official website explaining the new release. OpenAI's models were not trained on this content, so the only way to ask questions about it is to build a RAG chain.

The first thing to do is to load the blog content to our vector store. We will use beautiful soup to scrap the blog post. Then we will store it in a FAISS vector store. You can use any other vector store you prefer, common ones include CassandraDB or ChromaDB

In [ ]:
# retrieval chain

! pip install beautifulsoup4

In [ ]:
! pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/week-of-2-19-langchain-release-notes/")

docs = loader.load()

In [ ]:
docs

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter  import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)


In [ ]:
vectorstore = FAISS.from_documents(documents, embeddings)

## 3.2 Create a Context-Aware LLM Chain

Here we create a chain that will answer a question given a context. For now, we are passing the context manually, but in the next step we will pass in the documents fetched from the vector store we created above 👆

In [ ]:
# create chain for documents

from langchain.chains.combine_documents import create_stuff_documents_chain

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "what is langSmith",
    "context": [Document(page_content="langsmith is a debugging and testing framework used in LLM or Rag based applications.")]
})

## 3.2 Create the RAG Chain

RAG stands for Retrieval-Augmented Generation. This means that we will enrich the prompt that we send to the LLM. We will use with the documents that wil will retrieve from the vector store for this. LangChain comes with the function `create_retrieval_chain` that allows you to create one of these.

In [ ]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "what is new in langsmith"
})

In [ ]:
response['answer']

# 4.0 Create Conversational RAG Chain

Now we will create exactly the same thing as above, but we will have the AI assistant take the history of the conversation into account. In short, we will build the same chain as above but with we will take into account the previous messages in these two steps of the chain:

- When fetching the documents from the vector store. We will fetch documents related to the entire conversation and not just the latest message.
- When answering the question. We will send to the LLM the history of the conversation along the context and query.





## 4.1 Create a Conversation-Aware Retrieval Chain

This chain will return the documents related to the entire conversation and not just the latest message.

In [ ]:
# conversational retrieval chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Is there anything new about LangSmith?"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

## 4.2 Use Retrieval Chain together with Document Chain

Now we will create a document chain that contains a placeholder for the conversation history. This placeholder will be populated with the conversation history that we will pass as its value. We will the plug it together with the retriever chain we created above to have a conversational retrieval-augmented chain.

In [ ]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "What is langsmith?"
})

In [ ]:
response
# we only want the answer from the response so we can tap into the list and get the response answer

In [ ]:
# simulate conversation history

chat_history = [
    HumanMessage(content="Is there anything new about LangSmith?"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Tell me more about it!"
})

In [ ]:
response['answer']

# Conclusion

Congratulations for finishing this tutorial. For a more in-depth walk-through, you can go to the official [LangChain documentation](https://python.langchain.com/docs/get_started/quickstart).